In [6]:
import os
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed # 멀티스레딩을 위한 모듈
def delete_offline_augmented_images_multithreaded(project_dir, data_dir, num_threads=None):
    """
    오프라인 증강된 이미지를 멀티스레딩을 사용하여 효율적으로 삭제합니다.

    Args:
        cfg: 설정 객체 (cfg.data_dir을 포함해야 합니다).
        augmented_ids: 삭제할 이미지 파일 이름(경로 제외) 리스트.
        num_threads: 사용할 스레드 개수. None이면 CPU 코어 수에 따라 자동으로 결정됩니다.
                     (일반적으로 파일 I/O는 I/O 바운드 작업이므로 CPU 코어 수보다 많게 설정해도 유리할 수 있습니다.)
    """
    train_dir = os.path.join(project_dir, data_dir, 'train')
    deleted_count = 0
    wrong_filenames = []
    to_del = []
    for filename in os.listdir(train_dir):
        # 파일 이름이 "val"로 시작하고 파일인 경우
        if filename.startswith("val") and os.path.isfile(os.path.join(train_dir, filename)):
            file_path = os.path.join(train_dir, filename)
            to_del.append(file_path)

    # 스레드 풀 생성 (num_threads가 None이면 기본값으로 설정됨)
    # 파일 I/O 작업은 CPU 바운드라기보다는 I/O 바운드이므로, num_threads를 CPU 코어 수보다 높게 설정해도 좋습니다.
    # 하지만 너무 높게 설정하면 오버헤드가 발생할 수 있으니 적절한 값을 찾아야 합니다.
    # 일반적으로 넉넉하게 10-30 사이의 값을 시도해볼 수 있습니다.
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        # 각 파일 삭제 작업을 스레드 풀에 제출
        # executor.submit(함수, 인자1, 인자2, ...)
        futures = {executor.submit(_delete_single_image, train_dir, filename): filename for filename in to_del}

        # tqdm을 사용하여 진행 상황 표시
        # as_completed는 제출된 작업이 완료되는 순서대로 Future 객체를 반환합니다.
        for future in tqdm(as_completed(futures), total=len(to_del), desc="이미지 삭제 중"):
            filename = futures[future]
            try:
                # 작업 결과를 가져옴 (삭제 성공 여부)
                result = future.result()
                if result:
                    deleted_count += 1
                else:
                    wrong_filenames.append(filename) # 삭제 실패 시 기록
            except Exception as exc:
                # 스레드 내에서 예외 발생 시 처리
                print(f'{filename} 삭제 중 예외 발생: {exc}')
                wrong_filenames.append(filename)

    print(f"{deleted_count}개 이미지 제거 완료.")
    if wrong_filenames:
        print(f"삭제에 실패했거나 찾을 수 없는 파일: {len(wrong_filenames)}개")
        for wrong_file in wrong_filenames:
            print(f"  - {os.path.join(train_dir, wrong_file)}")

def _delete_single_image(train_dir, filename):
    """
    단일 이미지를 삭제하는 헬퍼 함수. 멀티스레딩 작업에 사용됩니다.
    """
    file_path = os.path.join(train_dir, filename)
    if os.path.exists(file_path):
        try:
            os.remove(file_path)
            return True  # 성공적으로 삭제됨
        except OSError as e:
            # 권한 문제 등으로 삭제 실패 시
            # print(f"파일 삭제 오류: {file_path} - {e}")
            return False
    else:
        # print("Wrong filename:", file_path) # tqdm 사용 시 print가 너무 많아질 수 있으므로 주석 처리하거나 로그에 기록
        return False # 파일이 존재하지 않음

In [12]:
project_root = '/data/ephemeral/home/upstageailab-cv-classification-cv_5/'
data_dirname = 'aug_data_500_new1'

delete_offline_augmented_images_multithreaded(project_root, data_dirname, num_threads=12)

이미지 삭제 중: 0it [00:00, ?it/s]

0개 이미지 제거 완료.
